In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
from ICARUS.database import Database

# CHANGE THIS TO YOUR DATABASE FOLDER
database_folder = "E:\\Icarus\\Data"

# Load the database
DB = Database(database_folder)
EXTERNAL_DB = DB.EXTERNAL_DB

# Open Database

In [ ]:
db3d = DB.vehicles_db
db3d.load_all_data()

# plane = db3d.get_vehicle("hermes")

### Select Plane

In [ ]:
from time import sleep

planenames: list[str] = DB.get_vehicle_names()
for i, planename in enumerate(planenames):
    print(f"{i} : `{planename}")
sleep(1)

indexes = [1]  # range(0,len(planenames))
plane = DB.get_vehicle(planenames[indexes[0]])
states = DB.get_vehicle_states(planenames[indexes[0]])
state = states['Unstick']
# dynplane = db3d.dyn_planes[planenames[index]]
planenames = [planenames[i] for i in indexes]

In [ ]:
DB.load_all_data()

# Add XFLR Polars

In [ ]:
from ICARUS.computation.solvers.XFLR5.polars import read_polars_3d
EXTERNAL_DB = DB.EXTERNAL_DB

for name in planenames:
    if name.startswith("XFLR"):
        continue

    if f"XFLR_{name}" not in planenames:
        try:
            import os

            XFLR5PLANEDIR: str = os.path.join(EXTERNAL_DB, f"{name}.txt")
            read_polars_3d(DB, XFLR5PLANEDIR, name)
            print(f"Imported XFLR polar for {name}")
            planenames.append(f"XFLR_{name}")
        except FileNotFoundError:
            print(f"No XFLR polar found for {name}")
            pass

# Airplane Visualization

In [ ]:
# plane.visualize()

# Polars

In [ ]:
# from ICARUS.computation.solvers.GenuVP.analyses.pertrubations import proccess_pertrubation_res_7
# proccess_pertrubation_res_7(plane, unstick)

In [ ]:
from ICARUS.visualization.airplane.db_polars import plot_airplane_polars

DB.vehicles_db.load_all_data()
solvers = [
     "GenuVP3 Potential",
     "GenuVP3 2D",
    # "GNVP3 ONERA",
    "GenuVP7 Potential",
    "GenuVP7 2D",
    #  "LSPT Potential",
    # "LSPT 2D",/'/
    "AVL"
]
axs, fig = plot_airplane_polars(
    planenames,
    solvers,
    plots=[["AoA", "CL"], ["AoA", "CD"], ["AoA", "Cm"]],
    size=(6, 7),
)

In [ ]:
from ICARUS.environment.definition import EARTH_ISA
import distinctipy

axs, fig = plot_airplane_polars(
    planenames,
    solvers,
    plots=[["AoA", "CL"]],
    size=(6, 7),
)


u_inf = 20
rho = EARTH_ISA.air_density
q_inf = 0.5 * rho * u_inf ** 2
visc = EARTH_ISA.air_dynamic_viscosity

sim_reyn = rho * u_inf * plane.main_wing.mean_aerodynamic_chord / visc

axs = axs.flatten()
airfoil = plane.airfoils[0]
airfoil = DB.get_airfoil(airfoil)
polar = DB.get_airfoil_polars(airfoil.name)

correction_factor = 1 + 2 / (plane.aspect_ratio)
print(correction_factor)

reynolds_nums = polar.reynolds_nums[::3]
colors = distinctipy.get_colors(len(reynolds_nums))
for i, reyn in enumerate(reynolds_nums):
    df = polar.get_reynolds_subtable(reyn)

    cl = df['CL'] 
    cl3d = cl / correction_factor
    aoa = df['AoA'] - plane.main_wing.orientation[0]

    # Set opacity to 0.5
    axs[0].plot(aoa, cl, label=f'{airfoil.name} {reyn:.2e}', alpha=0.8, linestyle = '--', color=colors[i])

# Clear all previous legends
fig.legend([], [], loc="lower right", ncol=2)
fig.suptitle(f"{airfoil.name} Cl vs AoA at Re = {sim_reyn:.2e}")
# Get the plot legend
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="lower left", ncol=2)


# reynolds = reynolds_nums[0]
# polar.plot_avl_drag(reynolds)

In [ ]:
from ICARUS.visualization.airplane.cg_investigation import cg_investigation

cg_investigation(
    planenames[0],
    solvers,
    size=(10, 7),
)

# Case Convergence

In [ ]:
from ICARUS.visualization.airplane.gnvp_convergence import plot_convergence

In [ ]:
angles2show = [0.0]  # All
# Only Works for unsteady solvers: LSPT, GNVP3, GNVP7
plot_convergence(
    plane.name, angles=angles2show, solvers=["All"], plot_error=False, size=(16, 7)
)

# Wake Visualization

In [ ]:
from ICARUS.visualization.airplane.gnvp_wake import plot_gnvp_wake
from ICARUS.database.utils import angle_to_case

In [ ]:
CASE: str = angle_to_case(4.0)
plot_gnvp_wake(7, plane, state, CASE, scale=True)

 # Loads Visualization

In [ ]:
from ICARUS.visualization.airplane.gnvp_strips import gnvp_strips_2d, gnvp_strips_3d
CASE: str = angle_to_case(-5.0)

NBs = [i + 1 for i in range(len(plane.surfaces))]
print(plane)
print("NBs to Visualize: ", NBs)

stripData = gnvp_strips_2d(plane, state, CASE, [NBs[0]], gnvp_version= 3, category="Wind")
stripData = gnvp_strips_3d(plane, state, CASE, [*NBs], gnvp_version= 3, category="Wind")

# Dynamics

In [ ]:
from ICARUS.flight_dynamics.state import State

states = DB.get_vehicle_states(plane)

for i, name in enumerate(states):
    print(f"{i}: {name}")

state: State = states["Unstick"]
# state.pertrubation_results

In [ ]:
# print(state)
state.stability_fd()
state.plot_eigenvalues()

In [ ]:
for dst in state.disturbances:
    print(dst)